In [2]:
import socket
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy

In [3]:
def le_dados_imu(N, func=None, endereco = "shaka", porta=1234):
    dd = []
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((endereco, porta))
        for i in range(N):
            data = s.recv(32)
            ax = int.from_bytes(data[0:2], byteorder='big', signed=True)
            ay = int.from_bytes(data[2:4], byteorder='big', signed=True) 
            az = int.from_bytes(data[4:6], byteorder='big', signed=True) 
            temp = int.from_bytes(data[6:8], byteorder='big', signed=True) 
            wx = int.from_bytes(data[8:10], byteorder='big', signed=True)
            wy = int.from_bytes(data[10:12], byteorder='big', signed=True)
            wz = int.from_bytes(data[12:14], byteorder='big', signed=True)
            mx = int.from_bytes(data[14:16], byteorder='big', signed=True)
            my = int.from_bytes(data[16:18], byteorder='big', signed=True)
            mz = int.from_bytes(data[18:20], byteorder='big', signed=True)
            t = int.from_bytes(data[-8:], byteorder='little', signed=False)
            dd.append([ax, ay, az, wx, wy, wz, mx, my, mz, temp, t])
            if func:
                func(i)
    return dd

In [4]:
res2x = np.loadtxt("res2x.txt")
scale = 0.000598
mag_scale = 0.001810
print(res2x)

[-181.4548672  -151.19968288  119.58441564    0.99810697    0.99971396
    0.98932928]


In [62]:
dados = le_dados_imu(6000, lambda i : print("Coletando dados: " + str(int(100*(i+1)/6000))+ "%", end="\r"))


ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente

In [25]:
plano1 = np.array ( dados ) [... ,0:3]
np.savetxt("plano1.txt",plano1)

In [27]:
plano2 = np.array ( dados ) [... ,0:3]
np.savetxt("plano2.txt",plano2)

In [29]:
plano3 = np.array ( dados ) [... ,0:3]
np.savetxt("plano3.txt",plano3)

In [31]:
plano4 = np.array ( dados ) [... ,0:3]
np.savetxt("plano4.txt",plano4)

In [12]:
plano5 = np.array ( dados ) [... ,0:3]
np.savetxt("plano5.txt",plano5)

In [15]:
inclinado = np.array ( dados ) [... ,0:3]
np.savetxt("inclinado.txt",inclinado)

In [5]:
plano1 = np.loadtxt("plano1.txt")
plano2 = np.loadtxt("plano2.txt")
plano3 = np.loadtxt("plano3.txt")
plano4 = np.loadtxt("plano4.txt")
plano5 = np.loadtxt("plano5.txt")
inclinado = np.loadtxt("inclinado.txt")
plano = np.concatenate((plano1,plano2,plano3,plano4))
Q = np.loadtxt("Q.txt")

In [6]:
def leitura_corr(x,y,z):
    return (x+res2x[0])*res2x[3]*scale,(y+res2x[1])*res2x[4]*scale,(z+res2x[2])*res2x[5]*scale

In [7]:
plano_corr = [leitura_corr(e[0],e[1],e[2]) for e in plano]


In [8]:
media_plano_corr = np.average(plano_corr,0)
norma = np.linalg.norm(media_plano_corr)
k = -media_plano_corr/norma
print(k)

[ 1.18577741e-01 -9.03299841e-04 -9.92944361e-01]


In [9]:
g1 = np.average(np.array([leitura_corr(e[0],e[1],e[2]) for e in plano5]),0)
g2 = np.average(np.array([leitura_corr(e[0],e[1],e[2]) for e in inclinado]),0)
d = g1-g2
j = np.cross(d,k)
j /= np.linalg.norm(j)
i = np.cross(j,k)
Q = np.vstack((i,j,k))
print(Q)
np.savetxt("Q.txt",Q)

[[ 9.92942367e-01  2.30880653e-03  1.18575402e-01]
 [ 2.18540728e-03 -9.99996927e-01  1.17069774e-03]
 [ 1.18577741e-01 -9.03299841e-04 -9.92944361e-01]]


In [10]:
import urllib.request
import json
with urllib.request.urlopen ("http://ip-api.com/json/") as req :
    geodata = json.loads(req.read().decode())
print(geodata)

{'status': 'success', 'country': 'Brazil', 'countryCode': 'BR', 'region': 'SP', 'regionName': 'São Paulo', 'city': 'São Paulo', 'zip': '01000', 'lat': -23.6301, 'lon': -46.6378, 'timezone': 'America/Sao_Paulo', 'isp': 'Universidade De SAO Paulo', 'org': 'Universidade De SAO Paulo', 'as': 'AS28571 UNIVERSIDADE DE SAO PAULO', 'query': '143.107.167.189'}


In [11]:

import ahrs
print(geodata['lat'],geodata['lon'])
wmm = ahrs.utils.WMM()
wmm.magnetic_field(geodata['lat'],geodata['lon'])
F, H, X, Y, Z, I, D = wmm.F, wmm.H, wmm.X, wmm.Y, wmm.Z, wmm.I, wmm.D
print(Z)

-23.6301 -46.6378
-14913.366078721565


In [20]:
dados = le_dados_imu(6000, lambda i : print("Coletando dados: " + str(int(100*(i+1)/6000))+ "%", end="\r"))

In [58]:
np.savetxt("dados_mag_rand.txt",dados)

In [74]:
np.savetxt("orientados.txt",dados)

In [21]:
np.savetxt("GyroParado.txt",dados)

In [ ]:

def costfuncmag (x , data ):
  cost = 0
  sxx = x[0]
  syy = x[1]
  szz = x[2]
  sxy = x[3]
  sxz = x[4]
  syz = x[5]
  hx = x[6]
  hy = x[7]
  hz = x[8]
  h = np.array(x[6:])
  S = np.array([[sxx, sxy, sxz],[sxy, syy, syz], [sxz, syz, szz]])
  data = np.array(data)
  sumg = 0
  for e in data:
    diff = e-h
    prod = S @ (diff.T)
    #print(S,diff,prod)
    norm = np.linalg.norm(prod)
    sumg += (norm - 1)**2
    G = sumg/len(data)
  return G

def grad_costfuncmag (x , data ):
    grad = np.zeros(9)
    sxx = x[0]
    syy = x[1]
    szz = x[2]
    sxy = x[3]
    sxz = x[4]
    syz = x[5]
    hx = x[6]
    hy = x[7]
    hz = x[8]
    S = np.array([[sxx, sxy, sxz],[sxy, syy, syz], [sxz, syz, szz]])
    h = np.array(x[6:])
    data = np.array(data)

    ## Atenção: conferir se os vetores grad[i] são linha ou coluna
    for e in data:
        Ui = S @ (e - h)
        #print(Ui.shape)
        Ri = np.sqrt(Ui.T @ Ui)
        #Ri = np.linalg.norm(Ui)
        '''
        grad[0] += (Ui/Ri) * (Ri-1) * np.array([e[0]-h[0], 0, 0]) #sxx
        grad[1] += (Ui/Ri) * (Ri-1) * np.array([0, e[1]-h[1], 0]) #syy
        grad[2] += (Ui/Ri) * (Ri-1) * np.array([0, 0, e[2]-h[2]]) #szz
        grad[3] += (Ui/Ri) * (Ri-1) * np.array([e[1]-h[1], e[0]-h[0], 0]) #sxy
        grad[4] += (Ui/Ri) * (Ri-1) * np.array([e[2]-h[2], 0, e[0]-h[0]]) #sxz
        grad[5] += (Ui/Ri) * (Ri-1) * np.array([0, e[2]-h[2], e[1]-h[1]]) #syz
        grad[6] += (Ui/Ri) * (Ri-1) * np.array([-sxx, -sxy, -sxz]) #hx
        grad[7] += (Ui/Ri) * (Ri-1) * np.array([-sxy, -syy, -syz]) #hy
        grad[8] += (Ui/Ri) * (Ri-1) * np.array([-sxz, -syz, -szz]) #hz
        '''
        grad[0] += np.array([e[0]-h[0], 0, 0]) @ Ui * (Ri-1)/Ri #sxx
        grad[1] += np.array([0, e[1]-h[1], 0]) @ Ui * (Ri-1)/Ri #syy
        grad[2] += np.array([0, 0, e[2]-h[2]]) @ Ui * (Ri-1)/Ri #szz
        grad[3] += np.array([e[1]-h[1], e[0]-h[0], 0]) @ Ui * (Ri-1)/Ri #sxy
        grad[4] += np.array([e[2]-h[2], 0, e[0]-h[0]]) @ Ui * (Ri-1)/Ri #sxz
        grad[5] += np.array([0, e[2]-h[2], e[1]-h[1]]) @ Ui * (Ri-1)/Ri #syz
        grad[6] += np.array([-sxx, -sxy, -sxz]) @ Ui * (Ri-1)/Ri #hx
        grad[7] += np.array([-sxy, -syy, -syz]) @ Ui * (Ri-1)/Ri #hy
        grad[8] += np.array([-sxz, -syz, -szz]) @ Ui * (Ri-1)/Ri #hz

    for i in range(len(grad)):
        grad[i] = grad[i] * 2 / len(data)
    return grad

dados_brutos = np.loadtxt("dados_mag_rand.txt")
captura = dados_brutos[:, 5:8]

lxi_max = np.max(captura[:, 0])
lxi_min = np.min(captura[:, 0])
lyi_max = np.max(captura[:, 1])
lyi_min = np.min(captura[:, 1])
lzi_max = np.max(captura[:, 2])
lzi_min = np.min(captura[:, 2])

sxx0 = 2/(lxi_max-lxi_min)
syy0 = 2/(lyi_max-lyi_min)
szz0 = 2/(lzi_max-lzi_min)
hx0 = (lxi_max+lxi_min)/2
hy0 = (lyi_max+lyi_min)/2
hz0 = (lzi_max+lzi_min)/2
sol_inic = np.array([sxx0, syy0, szz0, 0, 0, 0, hx0, hy0, hz0])

cal_mag = scipy.optimize.minimize(costfuncmag , sol_inic , captura , jac = grad_costfuncmag)

print(cal_mag.x)

np.savetxt("calmag.txt",cal_mag.x)

[ 1.84381064e-04  1.19548326e-02  1.11581080e-02 -3.60098284e-06
  1.40692531e-05  2.16073720e-04  1.41500773e+02 -1.09630285e+02
  9.59760175e+01]


In [ ]:

calmag = np.loadtxt("calmag.txt")
S = np.array([[calmag[0],calmag[3],calmag[4]],
                [calmag[3],calmag[1],calmag[5]],
                [calmag[4],calmag[5],calmag[2]]])
H = np.array([calmag[6],calmag[7],calmag[8]])

T = np.array([[0,1,0],
                [1,0,0],
                [0,0,-1]])

Sl = T@S
Hl = 

print(Sl)


[[-3.60098284e-06  1.19548326e-02  2.16073720e-04]
 [ 1.84381064e-04 -3.60098284e-06  1.40692531e-05]
 [-1.40692531e-05 -2.16073720e-04 -1.11581080e-02]]
